In [ ]:
import os
import shutil


def organize_files_in_folders(folder_paths):
    # Define the prefixes we're looking for
    prefixes = [
        "saebench_pythia-160m-deduped_width-2pow12_date-0108",
        # 'sae_bench_gemma-2-2b_vanilla_width-2pow16_date-1109'
    ]

    for folder in folder_paths:
        if not os.path.exists(folder):
            print(f"Folder {folder} does not exist. Skipping.")
            continue

        os.chdir(folder)  # Change to the target directory
        print(f"Processing folder: {folder}")

        # Create folders if they don't exist
        for prefix in prefixes:
            if not os.path.exists(prefix):
                os.makedirs(prefix)

        # Get all json files in the current directory
        files = [f for f in os.listdir(".") if f.endswith(".json")]

        # Move files to appropriate folders
        for file in files:
            for prefix in prefixes:
                if file.startswith(prefix):
                    shutil.move(file, os.path.join(prefix, file))
                    print(f"Moved {file} to {prefix}/")
                    break

        os.chdir("..")  # Return to the original directory


if __name__ == "__main__":
    # List of folders to process
    folder_paths = [
        "absorption",
        "autointerp",
        "core",
        "scr",
        "sparse_probing",
        "tpp",
        "unlearning",
    ]

    organize_files_in_folders(folder_paths)

In [ ]:
import shutil
import os


def copy_autointerp():
    source = "autointerp"
    destination = "autointerp_with_generations"

    if not os.path.exists(source):
        print(f"Source folder '{source}' does not exist. Aborting.")
        return

    if os.path.exists(destination):
        raise Exception(f"Destination folder '{destination}' already exists. Aborting.")

    try:
        shutil.copytree(source, destination)
        print(f"Copied '{source}' to '{destination}'.")
    except Exception as e:
        print(f"Failed to copy '{source}' to '{destination}': {e}")


if __name__ == "__main__":
    copy_autointerp()

In [ ]:
# purpose: remove the llm generations, which is over 99% of the file size

import os
import json


def process_json_files(directory):
    """Recursively process all JSON files in directory and its subdirectories"""
    count = 0
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".json"):
                filepath = os.path.join(root, file)
                try:
                    # Read the JSON file
                    with open(filepath, "r") as f:
                        data = json.load(f)

                    # Remove the key if it exists
                    if "eval_result_unstructured" in data:
                        del data["eval_result_unstructured"]
                        count += 1

                    # Write back the modified data
                    with open(filepath, "w") as f:
                        json.dump(data, f)

                    # print(f"Processed: {filepath}")

                except Exception as e:
                    print(f"Error processing {filepath}: {str(e)}")

    return count


# Process files starting from the current directory
starting_dir = "autointerp"
files_modified = process_json_files(starting_dir)
print(f"\nCompleted! Modified {files_modified} files.")

In [ ]:
import shutil
import os


def copy_autointerp():
    source = "core"
    destination = "core_with_feature_statistics"

    if not os.path.exists(source):
        print(f"Source folder '{source}' does not exist. Aborting.")
        return

    if os.path.exists(destination):
        raise Exception(f"Destination folder '{destination}' already exists. Aborting.")

    try:
        shutil.copytree(source, destination)
        print(f"Copied '{source}' to '{destination}'.")
    except Exception as e:
        print(f"Failed to copy '{source}' to '{destination}': {e}")


if __name__ == "__main__":
    copy_autointerp()

In [ ]:
# purpose: remove the feature statistics, which is over 99% of the file size

import os
import json


def process_json_files(directory):
    """Recursively process all JSON files in directory and its subdirectories"""
    count = 0
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".json"):
                filepath = os.path.join(root, file)
                try:
                    # Read the JSON file
                    with open(filepath, "r") as f:
                        data = json.load(f)

                    # Remove the key if it exists
                    if "eval_result_details" in data:
                        del data["eval_result_details"]
                        count += 1

                    # Write back the modified data
                    with open(filepath, "w") as f:
                        json.dump(data, f)

                    # print(f"Processed: {filepath}")

                except Exception as e:
                    print(f"Error processing {filepath}: {str(e)}")

    return count


# Process files starting from the current directory
starting_dir = "core"
files_modified = process_json_files(starting_dir)
print(f"\nCompleted! Modified {files_modified} files.")

In [ ]:
import json
import os
from tqdm import tqdm
import re

local_dir = "."
folders = [
    f"{local_dir}/absorption",
    f"{local_dir}/autointerp",
    f"{local_dir}/core",
    f"{local_dir}/scr",
    f"{local_dir}/sparse_probing",
    f"{local_dir}/tpp",
    f"{local_dir}/unlearning",
    f"{local_dir}/autointerp_with_generations",
    f"{local_dir}/core_with_feature_statistics",
]

def get_sae_bench_train_tokens(filename) -> int:
    """
    Calculate the number of training tokens based on the sae_release and sae_id.
    """
    if "saebench" not in filename:
        raise ValueError("This function is only for SAE Bench releases")

    batch_size = 2048

    if "step" not in filename:
        steps = 244140
        return steps * batch_size
    else:
        match = re.search(r"step_(\d+)", filename)
        if match:
            step = int(match.group(1))
            return step * batch_size
        else:
            raise ValueError("No step match found")

def process_file(filename: str):
    """
    Process a single file: load JSON, add training_tokens to sae_cfg_dict, and save it.
    """
    try:
        with open(filename, "r") as f:
            eval_results = json.load(f)

        if "sae_cfg_dict" not in eval_results:
            raise KeyError("sae_cfg_dict not found in the JSON file")
        
        eval_results["sae_cfg_dict"]["training_tokens"] = get_sae_bench_train_tokens(
            filename
        )

        with open(filename, "w") as f:
            json.dump(eval_results, f, indent=4)

        return True
    except (json.JSONDecodeError, KeyError, ValueError) as e:
        print(f"Error processing file {filename}: {e}")
        return False

def main():
    total = 0
    total_updated = 0

    for folder in tqdm(folders, desc="Processing folders"):
        if not os.path.exists(folder):
            print(f"Folder {folder} does not exist. Skipping.")
            continue

        for root, _, files in os.walk(folder):
            for file in files:
                if file.endswith(".json"):
                    total += 1
                    filepath = os.path.join(root, file)
                    if process_file(filepath):
                        total_updated += 1

    print(f"Total files: {total}, Total updated: {total_updated}")

if __name__ == "__main__":
    main()

In [ ]:
raise Exception("Stop here")

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

api.upload_folder(
    folder_path=".",
    path_in_repo="",
    repo_id="adamkarvonen/new_sae_bench_results",
    repo_type="dataset",
    allow_patterns="*eval_results.json",
    ignore_patterns=[".DS_Store"]
)